In [31]:
from classifiers import CoTrainingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import pickle

## Load Models

### TF-IDF

In [2]:
with open ("tfidf.pk", 'rb') as fin:
    tfidf = pickle.load(fin)

In [3]:
tfidf

TfidfVectorizer(max_df=5, ngram_range=(1, 3), stop_words='english')

### Word2Vec CBOW

In [4]:
cbow = Word2Vec.load("cbow.model")

In [5]:
cbow

### Word2Vec SkipGram

In [6]:
skg = Word2Vec.load("skg.model")

In [7]:
skg

## Importing Data

In [8]:
df = pd.read_csv("processed_train.csv")

In [9]:
df

,Unnamed: 0,id,tweet,subtask_a,subtask_b,subtask_c
0,0,86426,"['ask', 'native', 'american', 'take']",OFF,UNT,NaN
1,1,90194,"['go', 'home', '’', 'drunk', 'maga', 'trump', ...",OFF,TIN,IND
2,2,16820,"['amazon', 'investigating', 'chinese', 'employ...",NOT,NaN,NaN
3,3,62688,"['someone', 'shouldve', 'taken', 'piece', 'shi...",OFF,UNT,NaN
4,4,43605,"['obama', 'wanted', 'liberal', 'amp', 'illegal...",NOT,NaN,NaN
...,...,...,...,...,...,...
13235,13235,95338,"['sometimes', 'get', 'strong', 'vibe', 'people...",OFF,TIN,IND
13236,13236,67210,"['benidorm', '✅', 'creamfields', '✅', 'maga', ...",NOT,NaN,NaN
13237,13237,82921,"['report', 'garbage', 'dont', 'give', 'crap']",OFF,TIN,OTH
13238,13238,27429,['pussy'],OFF,UNT,NaN


### Removing Redundant Axes

In [10]:
df = df.drop(['Unnamed: 0', 'subtask_b', 'subtask_c', 'id'], axis=1)

In [11]:
df

,tweet,subtask_a
0,"['ask', 'native', 'american', 'take']",OFF
1,"['go', 'home', '’', 'drunk', 'maga', 'trump', ...",OFF
2,"['amazon', 'investigating', 'chinese', 'employ...",NOT
3,"['someone', 'shouldve', 'taken', 'piece', 'shi...",OFF
4,"['obama', 'wanted', 'liberal', 'amp', 'illegal...",NOT
...,...,...
13235,"['sometimes', 'get', 'strong', 'vibe', 'people...",OFF
13236,"['benidorm', '✅', 'creamfields', '✅', 'maga', ...",NOT
13237,"['report', 'garbage', 'dont', 'give', 'crap']",OFF
13238,['pussy'],OFF


### Renaming Columns

In [12]:
df = df.rename(columns={'subtask_a': 'Offensive'})

In [13]:
df

,tweet,Offensive
0,"['ask', 'native', 'american', 'take']",OFF
1,"['go', 'home', '’', 'drunk', 'maga', 'trump', ...",OFF
2,"['amazon', 'investigating', 'chinese', 'employ...",NOT
3,"['someone', 'shouldve', 'taken', 'piece', 'shi...",OFF
4,"['obama', 'wanted', 'liberal', 'amp', 'illegal...",NOT
...,...,...
13235,"['sometimes', 'get', 'strong', 'vibe', 'people...",OFF
13236,"['benidorm', '✅', 'creamfields', '✅', 'maga', ...",NOT
13237,"['report', 'garbage', 'dont', 'give', 'crap']",OFF
13238,['pussy'],OFF


### Converting Offensive to Numerical Value

In [14]:
def off(cls):
    if cls =='OFF':
        return 1
    return 0

In [15]:
df["Offensive"] = df["Offensive"].apply(off)

In [16]:
df

,tweet,Offensive
0,"['ask', 'native', 'american', 'take']",1
1,"['go', 'home', '’', 'drunk', 'maga', 'trump', ...",1
2,"['amazon', 'investigating', 'chinese', 'employ...",0
3,"['someone', 'shouldve', 'taken', 'piece', 'shi...",1
4,"['obama', 'wanted', 'liberal', 'amp', 'illegal...",0
...,...,...
13235,"['sometimes', 'get', 'strong', 'vibe', 'people...",1
13236,"['benidorm', '✅', 'creamfields', '✅', 'maga', ...",0
13237,"['report', 'garbage', 'dont', 'give', 'crap']",1
13238,['pussy'],1


In [17]:
df['tweet']

0                    ['ask', 'native', 'american', 'take']
1        ['go', 'home', '’', 'drunk', 'maga', 'trump', ...
2        ['amazon', 'investigating', 'chinese', 'employ...
3        ['someone', 'shouldve', 'taken', 'piece', 'shi...
4        ['obama', 'wanted', 'liberal', 'amp', 'illegal...
                               ...                        
13235    ['sometimes', 'get', 'strong', 'vibe', 'people...
13236    ['benidorm', '✅', 'creamfields', '✅', 'maga', ...
13237        ['report', 'garbage', 'dont', 'give', 'crap']
13238                                            ['pussy']
13239    ['spanishrevenge', 'v', 'justice', 'human', 'r...
Name: tweet, Length: 13240, dtype: object

## Fitting Word Models on Data

In [18]:
X_train, X_test, y_train, y_test = train_test_split(df['tweet'], df['Offensive'], stratify=df['Offensive'], random_state=0)

In [35]:
X_train

3759             ['friend', 'father', 'safe', 'talk', '”']
1623                                   ['president', 'im']
8650     ['depending', 'government', 'politician', 'fur...
7897     ['im', 'sure', 'really', 'low', 'carb', 'diet'...
365                   ['welcome', 'why', 'james', 'cryin']
                               ...                        
11365    ['nra', 'supported', 'gun', 'control', 'reagan...
8409     ['quite', 'good', 'faking', 'must', 'done', 'n...
7883                                             ['thank']
10443    ['abundantly', 'clear', 'lack', 'common', 'sen...
10972                 ['there', 'brexit', '👇', '🏻', 'url']
Name: tweet, Length: 9930, dtype: object

### TF-IDF

In [44]:
X_train_tf = tfidf.transform(X_train)

In [45]:
X_test_tf = tfidf.transform(X_test)

### Sentence Vectoriser

Finding the average vector for a given document

In [21]:
def sent_vect(sent, model):
    sent = eval(sent)
    vec = np.zeros(32)
    num = 0
    for w in sent:
        try:
            temp_vec = model.wv.get_vector(w)
            #print(temp_vec)
            vec = np.add(vec, temp_vec)
            num += 1
        except:
            pass
        
    return vec / np.sqrt(vec.dot(vec))

### CBOW

In [39]:
X_train_cbow = X_train.apply(sent_vect, model=cbow).sort_index()

In [40]:
X_train_cbow

0        [0.11678230874736859, 0.04701858112529256, 0.1...
1        [0.023887964134799274, -0.04221032997250825, 0...
4        [0.023089552048990755, -0.036891745280832305, ...
5        [0.08365669444110241, 0.024382291167563585, 0....
7        [0.1692174577116111, -0.028866439854166506, 0....
                               ...                        
13232    [0.05910713569141994, 0.03033006832724931, 0.2...
13235    [0.1772960903524207, 0.06816052902725968, 0.19...
13236    [-0.07846772515660165, -0.09172202196583056, 0...
13237    [0.1720133490472096, 0.0603833914465248, 0.166...
13239    [0.029200375665037092, -0.027191909181750278, ...
Name: tweet, Length: 9930, dtype: object

In [24]:
X_test_cbow = X_test.apply(sent_vect, model=cbow)

In [25]:
X_test_cbow

9240     [0.22905203337787333, 0.08973991493182583, 0.1...
13059    [0.06942990350221642, -0.016186961472640454, 0...
11361    [0.16402907662221408, 0.07372853363241103, 0.1...
3809     [0.07493143982202348, 0.02938246293545342, 0.1...
3565     [0.030048634939846088, -0.0433301868652566, 0....
                               ...                        
1126     [0.027757347007731643, 0.028976077059844983, 0...
7276     [0.14257477882976266, 0.10102919218974611, 0.1...
1426     [0.07895854153818001, 0.01998946279212703, 0.2...
12266    [0.11715303383211771, 0.07327943302894656, 0.1...
10325    [0.06463762043371678, 0.036110142431394196, 0....
Name: tweet, Length: 3310, dtype: object

### SkipGram

In [26]:
X_train_skg = X_train.apply(sent_vect, model=skg)

In [27]:
X_train_skg

3759     [0.1385263343767096, 0.019514727808935924, 0.1...
1623     [-0.08006676582537567, 0.14220038026696025, 0....
8650     [0.12745751758561877, 0.04590809994137651, 0.1...
7897     [0.148907211855406, 0.045357729192608165, 0.14...
365      [0.16997259111404198, 0.02170235144521857, 0.1...
                               ...                        
11365    [0.12765205081760966, -0.08407392444930872, 0....
8409     [0.0927742904159179, 0.10098635879091483, 0.18...
7883     [0.03132195112290745, -0.008785520980705315, 0...
10443    [0.12725105628106392, 0.0018205805003424804, 0...
10972    [-0.02001318080064111, -0.08916336317861015, 0...
Name: tweet, Length: 9930, dtype: object

In [28]:
X_test_skg = X_test.apply(sent_vect, model=skg)

In [29]:
X_train_skg

3759     [0.1385263343767096, 0.019514727808935924, 0.1...
1623     [-0.08006676582537567, 0.14220038026696025, 0....
8650     [0.12745751758561877, 0.04590809994137651, 0.1...
7897     [0.148907211855406, 0.045357729192608165, 0.14...
365      [0.16997259111404198, 0.02170235144521857, 0.1...
                               ...                        
11365    [0.12765205081760966, -0.08407392444930872, 0....
8409     [0.0927742904159179, 0.10098635879091483, 0.18...
7883     [0.03132195112290745, -0.008785520980705315, 0...
10443    [0.12725105628106392, 0.0018205805003424804, 0...
10972    [-0.02001318080064111, -0.08916336317861015, 0...
Name: tweet, Length: 9930, dtype: object

## Co - Training

### SVM vs SVM

In [32]:
svm2 = CoTrainingClassifier(SVC())

In [47]:
svm2.fit(X_train_tf, X_train_cbow, y_train)

KeyError: "Passing list-likes to .loc or [] with any missing labels is no longer supported. The following labels were missing: Int64Index([   2,    3,    6,    9,   23,\n            ...\n            9913, 9916, 9923, 9925, 9926],\n           dtype='int64', length=2482). See https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike"